## Final Capstone Project - The Battle of Neighborhoods (Week 1-2)

### Business Problem section

**Background**

According to Bloomberg News, the London Housing Market is in a rut. It is now facing a number of different headwinds, including the prospect of higher taxes and a warning from the Bank of England that U.K. home values could fall as much as 30 percent in the event of a disorderly exit from the European Union. More specifically, four overlooked cracks suggest that the London market may be in worse shape than many realize: hidden price falls, record-low sales, homebuilder exodus and tax hikes addressing overseas buyers of homes in England and Wales.

**Business Problem**

In this scenario, it is urgent to adopt machine learning tools in order to assist homebuyers clientele in London to make wise and effective decisions. As a result, the business problem we are currently posing is: how could we provide support to homebuyers clientele in to purchase a suitable real estate in London in this uncertain economic and financial scenario?

To solve this business problem, we are going to cluster London neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We will recommend profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

### Data section

Data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). The following fields comprise the address data included in Price Paid Data: Postcode; PAON Primary Addressable Object Name. Typically the house number or name; SAON Secondary Addressable Object Name. If there is a sub-building, for example, the building is divided into flats, there will be a SAON; Street; Locality; Town/City; District; County.

To explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we will access data through FourSquare API interface and arrange them as a dataframe for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we will be able to recommend profitable real estate investments.

### Methodology section

The Methodology section will describe the main components of our analysis and predication system. The Methodology section comprises four stages:

1. Collect Inspection Data
2. Explore and Understand Data
3. Data preparation and preprocessing 
4. Modeling

1. **Collect Inspection Data**

After importing the necessary libraries, we download the data from the HM Land Registry website as follows:

In [2]:
import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [3]:
#Read the data for examination (Source: http://landregistry.data.gov.uk/)
df_ppd = pd.read_csv("http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018.csv")

Before using data, we will have to explore and understand it.

**2. Explore and Understand Data**

We read the dataset that we collected from the HM Land Registry website into a pandas' data frame and display the first five rows of it as follows:

In [4]:
df_ppd.head(5)

,{726BF13A-993E-0A46-E053-6C04A8C01D0D},75000,2018-06-20 00:00,DL13 4BA,T,N,F,21,Unnamed: 8,HIGHFIELDS,TOW LAW,BISHOP AUCKLAND,COUNTY DURHAM,COUNTY DURHAM.1,A,A.1
0,{726BF13A-993F-0A46-E053-6C04A8C01D0D},115000,2018-07-13 00:00,DL17 9LB,S,N,F,4,NaN,THE LANE,WEST CORNFORTH,FERRYHILL,COUNTY DURHAM,COUNTY DURHAM,A,A
1,{726BF13A-9940-0A46-E053-6C04A8C01D0D},24000,2018-04-10 00:00,SR7 9AG,F,N,L,20B,NaN,WOODS TERRACE,MURTON,SEAHAM,COUNTY DURHAM,COUNTY DURHAM,A,A
2,{726BF13A-9941-0A46-E053-6C04A8C01D0D},56000,2018-06-22 00:00,DL5 5PS,T,N,F,6,NaN,HEILD CLOSE,NaN,NEWTON AYCLIFFE,COUNTY DURHAM,COUNTY DURHAM,A,A
3,{726BF13A-9942-0A46-E053-6C04A8C01D0D},220000,2018-05-25 00:00,DL16 7HE,D,N,F,25,NaN,BECKWITH CLOSE,KIRK MERRINGTON,SPENNYMOOR,COUNTY DURHAM,COUNTY DURHAM,A,A
4,{726BF13A-9943-0A46-E053-6C04A8C01D0D},58000,2018-05-09 00:00,DL14 6FH,F,N,L,23,NaN,AINTREE DRIVE,NaN,BISHOP AUCKLAND,COUNTY DURHAM,COUNTY DURHAM,A,A


In [5]:
df_ppd.shape

(1032408, 16)

Our dataset consists of over 700000 rows and 16 columns. We will now prepare and preprocess data accordingly.

**3. Data preparation and preprocessing**

At this stage, we prepare our dataset for the modeling process, opting for the most suitable machine learning algorithm for our scope. Accordingly, we perform the following steps:

 - Rename the column names
 - Format the date column
 - Sort data by date of sale
 - Select data only for the city of London
 - Make a list of street names in London
 - Calculate the street-wise average price of the property
 - Read the street-wise coordinates into a data frame, eliminating recurring word London from individual names
 - Join the data to find the coordinates of locations which fit into client's budget
 - Plot recommended locations on London map along with current market prices

In [6]:
# Assign meaningful column names
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

In [7]:
# Format the date column
df_ppd['Date_Transfer'] = df_ppd['Date_Transfer'].apply(pd.to_datetime)

# Delete all obsolete transactions which were done before 2016
df_ppd.drop(df_ppd[df_ppd.Date_Transfer.dt.year < 2016].index, inplace=True)

# Sort by Date of Sale
df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

In [8]:
df_ppd_london = df_ppd.query("Town_City == 'LONDON'")

# Make a list of street names in LONDON
streets = df_ppd_london['Street'].unique().tolist()

In [9]:
df_grp_price = df_ppd_london.groupby(['Street'])['Price'].mean().reset_index()

# Give meaningful names to the columns
df_grp_price.columns = ['Street', 'Avg_Price']

In [10]:
#Input your Budget's Upper Limit and Lower Limit - Find the locations df_grp_price which fits your budget
df_affordable = df_grp_price.query("(Avg_Price >= 2200000) & (Avg_Price <= 2500000)")

In [11]:
# Display the dataframe
df_affordable

,Street,Avg_Price
196,ALBION SQUARE,2450000.0
390,ANHALT ROAD,2435000.0
405,ANSDELL TERRACE,2250000.0
422,APPLEGARTH ROAD,2400000.0
857,BARONSMEAD ROAD,2375000.0
...,...,...
13736,WILFRED STREET,2410538.5
13762,WILLOW BRIDGE ROAD,2425000.0
13782,WILSON STREET,2257500.0
13811,WINCHENDON ROAD,2350000.0


In [13]:
import pandas as pd
import numpy as np
import datetime as DT
import hmac
from geopy.geocoders import Nominatim
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [14]:
for index, item in df_affordable.iterrows():
    print(f"index: {index}")
    print(f"item: {item}")
    print(f"item.Street only: {item.Street}")

index: 196
item: Street       ALBION SQUARE
Avg_Price         2.45e+06
Name: 196, dtype: object
item.Street only: ALBION SQUARE
index: 390
item: Street       ANHALT ROAD
Avg_Price      2.435e+06
Name: 390, dtype: object
item.Street only: ANHALT ROAD
index: 405
item: Street       ANSDELL TERRACE
Avg_Price           2.25e+06
Name: 405, dtype: object
item.Street only: ANSDELL TERRACE
index: 422
item: Street       APPLEGARTH ROAD
Avg_Price            2.4e+06
Name: 422, dtype: object
item.Street only: APPLEGARTH ROAD
index: 857
item: Street       BARONSMEAD ROAD
Avg_Price          2.375e+06
Name: 857, dtype: object
item.Street only: BARONSMEAD ROAD
index: 983
item: Street       BEAUCLERC ROAD
Avg_Price          2.48e+06
Name: 983, dtype: object
item.Street only: BEAUCLERC ROAD
index: 1105
item: Street       BELVEDERE DRIVE
Avg_Price           2.34e+06
Name: 1105, dtype: object
item.Street only: BELVEDERE DRIVE
index: 1218
item: Street       BICKENHALL STREET
Avg_Price           2.2085e+06
N

In [41]:
!pip install geopy
import geopy
from geopy.geocoders import Nominatim
geopy.geocoders.options.default_user_agent = "Foursquare"

In [42]:
!pip install geolocator

In [43]:
geolocator = Nominatim(user_agent="Foursquare")

In [44]:
df_affordable['city_coord'] = df_affordable['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

<ipython-input-44-0f5362039c03>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_affordable['city_coord'] = df_affordable['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))


In [45]:
df_affordable

,Street,Avg_Price,city_coord
196,ALBION SQUARE,2450000.0,"(-41.27375755, 173.28939323910353)"
390,ANHALT ROAD,2435000.0,"(29.805547, -98.475534)"
405,ANSDELL TERRACE,2250000.0,"(51.5000051, -0.1891537)"
422,APPLEGARTH ROAD,2400000.0,"(40.271236, -74.4711104)"
857,BARONSMEAD ROAD,2375000.0,"(51.4773147, -0.239457)"
...,...,...,...
13736,WILFRED STREET,2410538.5,"(54.9466099, -1.4586081)"
13762,WILLOW BRIDGE ROAD,2425000.0,"(53.6407507, -1.2006118)"
13782,WILSON STREET,2257500.0,"(45.7004857, -121.52187527526078)"
13811,WINCHENDON ROAD,2350000.0,"(42.620654, -72.074978)"


In [46]:
df_affordable[['Latitude', 'Longitude']] = df_affordable['city_coord'].apply(pd.Series)

C:\Users\DELL\anaconda3\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [47]:
df_affordable

,Street,Avg_Price,city_coord,Latitude,Longitude
196,ALBION SQUARE,2450000.0,"(-41.27375755, 173.28939323910353)",-41.273758,173.289393
390,ANHALT ROAD,2435000.0,"(29.805547, -98.475534)",29.805547,-98.475534
405,ANSDELL TERRACE,2250000.0,"(51.5000051, -0.1891537)",51.500005,-0.189154
422,APPLEGARTH ROAD,2400000.0,"(40.271236, -74.4711104)",40.271236,-74.471110
857,BARONSMEAD ROAD,2375000.0,"(51.4773147, -0.239457)",51.477315,-0.239457
...,...,...,...,...,...
13736,WILFRED STREET,2410538.5,"(54.9466099, -1.4586081)",54.946610,-1.458608
13762,WILLOW BRIDGE ROAD,2425000.0,"(53.6407507, -1.2006118)",53.640751,-1.200612
13782,WILSON STREET,2257500.0,"(45.7004857, -121.52187527526078)",45.700486,-121.521875
13811,WINCHENDON ROAD,2350000.0,"(42.620654, -72.074978)",42.620654,-72.074978


In [48]:
df = df_affordable.drop(columns=['city_coord'])

In [49]:
df

,Street,Avg_Price,Latitude,Longitude
196,ALBION SQUARE,2450000.0,-41.273758,173.289393
390,ANHALT ROAD,2435000.0,29.805547,-98.475534
405,ANSDELL TERRACE,2250000.0,51.500005,-0.189154
422,APPLEGARTH ROAD,2400000.0,40.271236,-74.471110
857,BARONSMEAD ROAD,2375000.0,51.477315,-0.239457
...,...,...,...,...
13736,WILFRED STREET,2410538.5,54.946610,-1.458608
13762,WILLOW BRIDGE ROAD,2425000.0,53.640751,-1.200612
13782,WILSON STREET,2257500.0,45.700486,-121.521875
13811,WINCHENDON ROAD,2350000.0,42.620654,-72.074978


In [51]:
address = 'London, UK'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London City are 51.5073219, -0.1276474.


In [52]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(df['Latitude'], df['Longitude'], df['Avg_Price'], df['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [53]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'YDXPDM2N0AJADNXBS5O2JXOYVW5RH0JBAMHVAC03HVOWFVDI' # your Foursquare ID
CLIENT_SECRET = 'JZ1DYEUO2IRQF2JII1442VWPYQMIRA5PW3OXBDLW4S5S2LLD' # your Foursquare Secret
ACCESS_TOKEN = 'U4OX31M0GHKJIQ0FILUE3C543TQC1IEQZYAQQFBVJMRZFY0Z' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YDXPDM2N0AJADNXBS5O2JXOYVW5RH0JBAMHVAC03HVOWFVDI
CLIENT_SECRET:JZ1DYEUO2IRQF2JII1442VWPYQMIRA5PW3OXBDLW4S5S2LLD


We can now proceed to the Modeling phase. We will analyze neighborhoods to recommend real estates where home buyers can make a real estate investment. We will then recommend profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores

**4. Modeling**

After exploring the dataset and gaining insights into it, we are ready to use the clustering methodology to analyze real estates. We will use the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate

In [55]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [56]:
# Run the above function on each location and create a new dataframe called location_venues and display it.
location_venues = getNearbyVenues(names=df['Street'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

ALBION SQUARE
ANHALT ROAD
ANSDELL TERRACE
APPLEGARTH ROAD
BARONSMEAD ROAD
BEAUCLERC ROAD
BELVEDERE DRIVE
BICKENHALL STREET
BIRCHLANDS AVENUE
BRAMPTON GROVE
BRIARDALE GARDENS
BROOKWAY
BURY WALK
CALLCOTT STREET
CAMPDEN HILL ROAD
CAMPION ROAD
CANNING PLACE
CARLISLE ROAD
CARLTON GARDENS
CARLYLE COURT
CHALCOT SQUARE
CHARLES LANE
CHELSEA CRESCENT
CHESTER CLOSE NORTH
CHEYNE COURT
CHEYNE ROW
CHISWICK MALL
CITY ROAD
CLARENDON STREET
CLONCURRY STREET
COLBECK MEWS
COLLEGE CRESCENT
CORNWALL TERRACE MEWS
COURT LANE GARDENS
CRESCENT GROVE
DALEBURY ROAD
DEWHURST ROAD
DORIA ROAD
DOWNSHIRE HILL
DUCHESS WALK
ECCLESTON SQUARE MEWS
EGBERT STREET
EGERTON PLACE
ELM PARK ROAD
FLORAL STREET
FRANK DIXON WAY
FULTON MEWS
GERARD ROAD
GERRARD ROAD
GIRDLERS ROAD
GLOUCESTER CRESCENT
GORDON PLACE
GRAFTON SQUARE
GRAHAM TERRACE
HARMAN DRIVE
HARRIS STREET
HAVANNAH STREET
HAZLEWELL ROAD
HEREFORD MEWS
HERONDALE AVENUE
HIGHWOOD HILL
HILLGATE PLACE
HOLLYCROFT AVENUE
HOLLYWOOD MEWS
HONEYWELL ROAD
HORTENSIA ROAD
HOXTON SQUARE

In [57]:
location_venues

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ALBION SQUARE,-41.273758,173.289393,The Free House,-41.273340,173.287364,Bar
1,ALBION SQUARE,-41.273758,173.289393,The Indian Cafe,-41.273308,173.286530,Indian Restaurant
2,ALBION SQUARE,-41.273758,173.289393,Queen's Gardens,-41.273671,173.291383,Park
3,ALBION SQUARE,-41.273758,173.289393,The Bridge Street Collective,-41.272520,173.285517,Café
4,ALBION SQUARE,-41.273758,173.289393,Urban,-41.274355,173.286317,New American Restaurant
...,...,...,...,...,...,...,...
4549,WILSON STREET,45.700486,-121.521875,Lake Taco,45.703056,-121.524445,Mexican Restaurant
4550,WILSON STREET,45.700486,-121.521875,The Shed,45.700849,-121.523560,Bar
4551,WILSON STREET,45.700486,-121.521875,Juanita's Marketa,45.700125,-121.524078,Grocery Store
4552,WILSON STREET,45.700486,-121.521875,Monagon's Pancake House,45.699474,-121.524643,Breakfast Spot


In [58]:
location_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
ALBION SQUARE,25,25,25,25,25,25
ANSDELL TERRACE,65,65,65,65,65,65
APPLEGARTH ROAD,5,5,5,5,5,5
BARONSMEAD ROAD,15,15,15,15,15,15
BEAUCLERC ROAD,1,1,1,1,1,1
...,...,...,...,...,...,...
WELLINGTON STREET,79,79,79,79,79,79
WESTMORELAND PLACE,16,16,16,16,16,16
WHITFIELD STREET,8,8,8,8,8,8


In [59]:
# get the List of Unique Categories
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))

There are 340 uniques categories.


In [60]:
location_venues.shape

(4554, 7)

In [61]:
# one hot encoding
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_onehot['Street'] = location_venues['Street'] 

# move street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Street,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Service,American Restaurant,Antique Shop,Arcade,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
london_grouped = venues_onehot.groupby('Street').mean().reset_index()
london_grouped

,Street,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Service,American Restaurant,Antique Shop,Arcade,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,ALBION SQUARE,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,ANSDELL TERRACE,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.015385,0.0,0.0,0.0
2,APPLEGARTH ROAD,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,BARONSMEAD ROAD,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,BEAUCLERC ROAD,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,WELLINGTON STREET,0.000000,0.0,0.0,0.0,0.0,0.0,0.012658,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
139,WESTMORELAND PLACE,0.000000,0.0,0.0,0.0,0.0,0.0,0.062500,0.0,0.0,...,0.0,0.062500,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
140,WHITFIELD STREET,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
141,WILFRED STREET,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [63]:
london_grouped.shape

(143, 341)

In [64]:
# What are the top 5 venues/facilities nearby profitable real estate investments?#

num_top_venues = 5

for hood in london_grouped['Street']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALBION SQUARE----
               venue  freq
0               Café  0.20
1         Restaurant  0.08
2                Pub  0.08
3                Bar  0.08
4  Indian Restaurant  0.08


----ANSDELL TERRACE----
                venue  freq
0               Hotel  0.08
1           Juice Bar  0.06
2          Restaurant  0.06
3  Italian Restaurant  0.06
4   Indian Restaurant  0.06


----APPLEGARTH ROAD----
                  venue  freq
0     Convenience Store   0.4
1      Storage Facility   0.2
2  Fast Food Restaurant   0.2
3            Donut Shop   0.2
4                   ATM   0.0


----BARONSMEAD ROAD----
             venue  freq
0      Coffee Shop  0.07
1      Sports Club  0.07
2             Park  0.07
3  Nature Preserve  0.07
4        Bookstore  0.07


----BEAUCLERC ROAD----
                           venue  freq
0        Health & Beauty Service   1.0
1                            ATM   0.0
2                        Parking   0.0
3  Paper / Office Supplies Store   0.0
4                   

In [65]:
# Define a function to return the most common venues/facilities nearby real estate investments#

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [66]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [67]:
# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = london_grouped['Street']

for ind in np.arange(london_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

In [68]:
venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALBION SQUARE,Café,Bar,Indian Restaurant,Pub,Restaurant,Asian Restaurant,Museum,French Restaurant,Fish & Chips Shop,New American Restaurant
1,ANSDELL TERRACE,Hotel,Restaurant,Indian Restaurant,Juice Bar,Italian Restaurant,Café,Pub,Clothing Store,Grocery Store,English Restaurant
2,APPLEGARTH ROAD,Convenience Store,Fast Food Restaurant,Donut Shop,Storage Facility,Escape Room,Event Space,Exhibit,Factory,Falafel Restaurant,Farm
3,BARONSMEAD ROAD,Indie Movie Theater,Thai Restaurant,Convenience Store,Pub,Coffee Shop,Restaurant,Park,Farmers Market,Nature Preserve,Movie Theater
4,BEAUCLERC ROAD,Health & Beauty Service,Yoga Studio,Egyptian Restaurant,English Restaurant,Escape Room,Event Space,Exhibit,Factory,Falafel Restaurant,Farm


In [69]:
venues_sorted.shape

(143, 11)

In [70]:
london_grouped.shape

(143, 341)

In [71]:
london_grouped=df

After our inspection of venues/facilities/amenities nearby the most profitable real estate investments in London, we could begin by clustering properties by venues/facilities/amenities nearby

In [72]:
#Distribute in 5 Clusters

# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([1, 4, 2, 4, 0, 1, 0, 2, 2, 1, 4, 4, 1, 0, 0, 1, 4, 2, 1, 3, 3, 4,
       1, 1, 2, 4, 3, 1, 2, 4, 0, 4, 0, 0, 3, 4, 4, 0, 2, 1, 0, 3, 2, 3,
       2, 2, 3, 2, 2, 4])

In [73]:
#Dataframe to include Clusters

london_grouped_clustering=df
london_grouped_clustering.head()

,Street,Avg_Price,Latitude,Longitude
196,ALBION SQUARE,2450000.0,-41.273758,173.289393
390,ANHALT ROAD,2435000.0,29.805547,-98.475534
405,ANSDELL TERRACE,2250000.0,51.500005,-0.189154
422,APPLEGARTH ROAD,2400000.0,40.271236,-74.471110
857,BARONSMEAD ROAD,2375000.0,51.477315,-0.239457


In [74]:
london_grouped_clustering.shape

(159, 4)

In [75]:
df.shape

(159, 4)

In [76]:
london_grouped_clustering.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [77]:
df.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [78]:
# add clustering labels
london_grouped_clustering['Cluster Labels'] = kmeans.labels_

# merge london_grouped with london_data to add latitude/longitude for each neighborhood
london_grouped_clustering = london_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')

london_grouped_clustering.head(30) # check the last columns!

,Street,Avg_Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
196,ALBION SQUARE,2.450000e+06,-41.273758,173.289393,1,Café,Bar,Indian Restaurant,Pub,Restaurant,Asian Restaurant,Museum,French Restaurant,Fish & Chips Shop,New American Restaurant
390,ANHALT ROAD,2.435000e+06,29.805547,-98.475534,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
405,ANSDELL TERRACE,2.250000e+06,51.500005,-0.189154,2,Hotel,Restaurant,Indian Restaurant,Juice Bar,Italian Restaurant,Café,Pub,Clothing Store,Grocery Store,English Restaurant
422,APPLEGARTH ROAD,2.400000e+06,40.271236,-74.471110,4,Convenience Store,Fast Food Restaurant,Donut Shop,Storage Facility,Escape Room,Event Space,Exhibit,Factory,Falafel Restaurant,Farm
857,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457,0,Indie Movie Theater,Thai Restaurant,Convenience Store,Pub,Coffee Shop,Restaurant,Park,Farmers Market,Nature Preserve,Movie Theater
983,BEAUCLERC ROAD,2.480000e+06,30.207887,-81.628209,1,Health & Beauty Service,Yoga Studio,Egyptian Restaurant,English Restaurant,Escape Room,Event Space,Exhibit,Factory,Falafel Restaurant,Farm
1105,BELVEDERE DRIVE,2.340000e+06,38.240016,-77.392041,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1218,BICKENHALL STREET,2.208500e+06,51.521201,-0.158908,2,Pizza Place,Restaurant,Sandwich Place,Breakfast Spot,Movie Theater,Greek Restaurant,Garden,Café,Museum,Bar
1256,BIRCHLANDS AVENUE,2.217000e+06,51.448414,-0.160457,2,Pub,Bakery,Pizza Place,Brewery,French Restaurant,Chinese Restaurant,Coffee Shop,Lake,Farm,Escape Room
1556,BRAMPTON GROVE,2.456875e+06,51.570365,-0.283394,1,Men's Store,Middle Eastern Restaurant,Lake,Bar,Food,Farmers Market,Escape Room,Event Space,Exhibit,Food Court


In [79]:
# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_grouped_clustering['Latitude'], london_grouped_clustering['Longitude'], london_grouped_clustering['Street'], london_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [80]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 0, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
857,2375000.0,Indie Movie Theater,Thai Restaurant,Convenience Store,Pub,Coffee Shop,Restaurant,Park,Farmers Market,Nature Preserve,Movie Theater
1105,2340000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2071,2375000.0,Pub,Park,Yoga Studio,Hotel,Indian Restaurant,Grocery Store,Movie Theater,Gastropub,Garden,Bookstore
2132,2379652.7,Pub,Coffee Shop,Hotel,Park,Café,Indian Restaurant,Pizza Place,Fast Food Restaurant,Bookstore,Italian Restaurant
2946,2367500.0,Hotel,Pub,Garden,Café,Italian Restaurant,Coffee Shop,Mediterranean Restaurant,Chinese Restaurant,Bar,Gym / Fitness Center


In [81]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 1, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
196,2450000.0,Café,Bar,Indian Restaurant,Pub,Restaurant,Asian Restaurant,Museum,French Restaurant,Fish & Chips Shop,New American Restaurant
983,2480000.0,Health & Beauty Service,Yoga Studio,Egyptian Restaurant,English Restaurant,Escape Room,Event Space,Exhibit,Factory,Falafel Restaurant,Farm
1556,2456875.0,Men's Store,Middle Eastern Restaurant,Lake,Bar,Food,Farmers Market,Escape Room,Event Space,Exhibit,Food Court
1983,2492500.0,Supermarket,English Restaurant,Park,Fast Food Restaurant,Discount Store,Café,Rental Car Location,Coffee Shop,Gym,American Restaurant
2139,2461000.0,Spa,Windmill,Soccer Stadium,Bus Station,Yoga Studio,Fast Food Restaurant,Escape Room,Event Space,Exhibit,Factory


In [82]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 2, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
405,2250000.0,Hotel,Restaurant,Indian Restaurant,Juice Bar,Italian Restaurant,Café,Pub,Clothing Store,Grocery Store,English Restaurant
1218,2208500.0,Pizza Place,Restaurant,Sandwich Place,Breakfast Spot,Movie Theater,Greek Restaurant,Garden,Café,Museum,Bar
1256,2217000.0,Pub,Bakery,Pizza Place,Brewery,French Restaurant,Chinese Restaurant,Coffee Shop,Lake,Farm,Escape Room
2228,2200000.0,Hotel,Yoga Studio,Fast Food Restaurant,English Restaurant,Escape Room,Event Space,Exhibit,Factory,Falafel Restaurant,Farm
2640,2250000.0,Café,Pub,Coffee Shop,French Restaurant,English Restaurant,Garden,Grocery Store,Plaza,Women's Store,Bagel Shop


In [83]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 3, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2245,2.300000e+06,Farm,Yoga Studio,Fast Food Restaurant,English Restaurant,Escape Room,Event Space,Exhibit,Factory,Falafel Restaurant,Farmers Market
2408,2.286679e+06,Café,Pub,Bar,Coffee Shop,Italian Restaurant,French Restaurant,Park,Furniture / Home Store,Greek Restaurant,Gourmet Shop
2688,2.287500e+06,Pub,Brewery,Gift Shop,Music Venue,Harbor / Marina,Reservoir,Art Museum,Farmers Market,Escape Room,Event Space
3379,2.298000e+06,Hotel,Yoga Studio,Fast Food Restaurant,English Restaurant,Escape Room,Event Space,Exhibit,Factory,Falafel Restaurant,Farm
4287,2.265000e+06,Pub,Yoga Studio,Fast Food Restaurant,English Restaurant,Escape Room,Event Space,Exhibit,Factory,Falafel Restaurant,Farm


In [84]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 4, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
390,2435000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
422,2400000.0,Convenience Store,Fast Food Restaurant,Donut Shop,Storage Facility,Escape Room,Event Space,Exhibit,Factory,Falafel Restaurant,Farm
1635,2397132.0,Park,Coffee Shop,Grocery Store,Breakfast Spot,Cricket Ground,Fast Food Restaurant,Escape Room,Event Space,Exhibit,Factory
1800,2400000.0,Electronics Store,Home Service,Fast Food Restaurant,English Restaurant,Escape Room,Event Space,Exhibit,Factory,Falafel Restaurant,Farm
2161,2425000.0,Hotel,Juice Bar,Indian Restaurant,Italian Restaurant,Restaurant,Mediterranean Restaurant,Garden,Chinese Restaurant,French Restaurant,Pub


### Results and Discussion section

First of all, even though the London Housing Market may be in a rut, it is still an "ever-green" for business affairs.

We may discuss our results under two main perspectives.

First, we may examine them according to neighborhoods/London areas. It is interesting to note that, although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair.

Second, we may analyze our results according to the five clusters we have produced. Even though, all clusters could praise an optimal range of facilities and amenities, we have found two main patterns. The first pattern we are referring to, i.e. Clusters 0, 2 and 4, may target home buyers prone to live in 'green' areas with parks, waterfronts. Instead, the second pattern we are referring to, i.e. Clusters 1 and 3, may target individuals who love pubs, theatres and soccer.

### Conclusion

To sum up, according to Bloomberg News, the London Housing Market is in a rut. It is now facing a number of different headwinds, including the prospect of higher taxes and a warning from the Bank of England that U.K. home values could fall as much as 30 percent in the event of a disorderly exit from the European Union. In this scenario, it is urgent to adopt machine learning tools in order to assist homebuyers clientele in London to make wise and effective decisions. As a result, the business problem we were posing was: how could we provide support to homebuyers clientele in to purchase a suitable real estate in London in this uncertain economic and financial scenario?

To solve this business problem, we clustered London neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We recommended profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.


First, we gathered data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). Moreover, to explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we accessed data through FourSquare API interface and arranged them as a data frame for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we were able to recommend profitable real estate investments.

Second, The Methodology section comprised four stages: 1. Collect Inspection Data; 2. Explore and Understand Data; 3. Data preparation and preprocessing; 4. Modeling. In particular, in the modeling section, we used the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate.

Finally, we drew the conclusion that even though the London Housing Market may be in a rut, it is still an "ever-green" for business affairs. We discussed our results under two main perspectives. First, we examined them according to neighborhoods/London areas. although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair. Second, we analyzed our results according to the five clusters we produced. While Clusters 0, 2 and 4 may target home buyers prone to live in 'green' areas with parks, waterfronts, Clusters 1 and 3 may target individuals who love pubs, theatres and soccer.